# Importing libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime
import statsmodels.api as sm

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [3]:
hub = pd.read_csv("ind_nifty50list (1).csv",index_col=0)
tickers=[]
for i in range(len(hub)):
    tickers.append(hub['Symbol'][i] + str(".NS"))

In [ ]:
start= datetime(2015,1,1)
end = datetime.today()

ohlc_daily  = {}

for i in range(len(tickers)):
    ohlc_daily[tickers[i]] = pdr.get_data_yahoo(tickers[i],start = start , end = end)

In [144]:
def optimal_weights(df):
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    ef = EfficientFrontier(mu, S,weight_bounds=(0, 1))
    raw_weights = ef.max_sharpe()
    raw_weights = list(raw_weights.values())
    weights = np.array(raw_weights)
    return weights

In [9]:
def portfolio_returns(weights,df):
    portfolio = pd.DataFrame()
    portfolio['ret'] = np.sum(weights*df,axis=1)
    return portfolio

In [98]:
def benchmark_performance(start,end):
    NIFTY = pdr.get_data_yahoo('^NSEI',start = start, end = end)
    benchmark = pd.DataFrame()    
    benchmark['ret'] = (NIFTY['Adj Close'] - NIFTY['Adj Close'].shift(1))/ NIFTY['Adj Close'].shift(1)
#     benchmark['ret'] = benchmark['ret'].resample('M').mean()

    benchmark_portfolio = performance_analysis(benchmark,window=3,rf=0.025,leverage=1)
    return benchmark_portfolio,NIFTY

In [136]:
def performance_analysis(df,window,rf,leverage):    
    
    df['cum_return'] = (1+df['ret']*leverage).cumprod()
    Cumm_Ret = (df['cum_return'][-1]-1)
    
    n = len(df)/(12)   
    CAGR = ((df["cum_return"][-1])**(1/n) - 1)
    
    df['rolling_volatility'] = df['ret'].rolling(window).std()
    volatility =  df['ret'].std()*np.sqrt(12)
    
    df['rolling_sharpe'] = df['ret'].rolling(window).mean()/df['ret'].rolling(window).std()*np.sqrt(window)
    sr = (CAGR - rf)/volatility
    
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    
    df["Year"] = df.index.map(lambda x: x.year)
    df["Month"] = df.index.map(lambda x: x.strftime("%b"))
    pt = df.pivot_table(index="Month",columns="Year",values="ret", aggfunc="sum").fillna(0)
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    pt = pt.reindex(months)    
    sns.heatmap(pt, annot=True, cmap="RdYlGn")
    
    performance_df = pd.DataFrame([Cumm_Ret*100,CAGR*100,volatility,sr,max_dd*100],index=["Cumm_Return","Annual_Return","Volatility","Sharpe Ratio","Max Drawdown"])      
    print(performance_df)

    return df

In [ ]:
NIFTY_50_perf,NIFTY_50 = benchmark_performance(start=datetime(2015,6,30),end=datetime.today())
NIFTY_50_perf['cum_return'].plot(figsize=(10,8),grid=True)

In [ ]:
ohlc_monthly = {}
for ticker in tickers:
    ohlc_monthly[ticker]= ohlc_daily[ticker]['Adj Close'].resample('M').mean()

monthly_database = pd.DataFrame()
for ticker in tickers:
    monthly_database = pd.concat([monthly_database,ohlc_monthly[ticker]],axis=1)
monthly_database.columns= tickers

monthly_database_ret = monthly_database.pct_change()
monthly_database_ret.dropna(inplace=True)
monthly_database= monthly_database.drop(monthly_database.index[0])


In [ ]:
def Dynamic_Portfolio_Optimizer(df,df_ret):
    dynamic_portfolio = []
    for i in range(4,len(df)):
        weights = optimal_weights(monthly_database[i-4:i])
        test_portfolio = portfolio_returns(weights,df_ret[i:i+1])
        dynamic_portfolio.append(test_portfolio['ret'])
#         print(i,dynamic_portfolio)
    return dynamic_portfolio

In [ ]:
Dynamic_portfolcio = Dynamic_Portfolio_Optimizer(df=monthly_database,df_ret=monthly_database_ret)
Dynamic_portfolio_arr = np.array(Dynamic_portfolio)
Dynamic_portfolio_df = pd.DataFrame(Dynamic_portfolio_arr)

Dynamic_portfolio_df.columns = ['ret']
Dynamic_portfolio_df['date'] = pd.date_range(start=datetime(2015,6,30), periods=len(Dynamic_portfolio_df), freq='M ')
Dynamic_portfolio_df = Dynamic_portfolio_df.set_index('date')

In [ ]:
Dynamic_portfolio_perf = performance_analysis(df=Dynamic_portfolio_df,window=3,rf=0.25,leverage=1)
Dynamic_portfolio_perf['cum_return'].plot(figsize=(10,8),grid=True)

# CAPM Model

In [128]:
X = Dynamic_portfolio_df['ret']
X = X.drop(X.index[0])
y1 = pdr.get_data_yahoo('^NSEI',start = datetime(2015,6,30), end = datetime.today())  
y = y1['Adj Close'].pct_change()
y = y.resample('M').mean()
y = y.drop(y.index[0])

# Add a constant to the independent value
X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y, X1)

# fit model and print results
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              Adj Close   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     10.39
Date:                Fri, 13 Mar 2020   Prob (F-statistic):            0.00213
Time:                        21:36:56   Log-Likelihood:                 265.14
No. Observations:                  57   AIC:                            -526.3
Df Residuals:                      55   BIC:                            -522.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002      0.000     -0.659      0.5

In [127]:
print(len(X),len(y))

57 57


In [126]:
y

Date
2015-07-31    0.000882
2015-08-31   -0.003115
2015-09-30   -0.000063
2015-10-31    0.000754
2015-11-30   -0.000879
2015-12-31    0.000093
2016-01-31   -0.002531
2016-02-29   -0.003683
2016-03-31    0.005177
2016-04-30    0.000841
2016-05-31    0.001803
2016-06-30    0.000735
2016-07-31    0.002091
2016-08-31    0.000829
2016-09-30   -0.000973
2016-10-31    0.000201
2016-11-30   -0.002269
2016-12-31   -0.000182
2017-01-31    0.002156
2017-02-28    0.001937
2017-03-31    0.001499
2017-04-30    0.000797
2017-05-31    0.001541
2017-06-30   -0.000493
2017-07-31    0.002717
2017-08-31   -0.000733
2017-09-30   -0.000605
2017-10-31    0.002732
2017-11-30   -0.000464
2017-12-31    0.001485
2018-01-31    0.002210
2018-02-28   -0.002575
2018-03-31   -0.001897
2018-04-30    0.002878
2018-05-31    0.000006
2018-06-30   -0.000080
2018-07-31    0.002660
2018-08-31    0.001355
2018-09-30   -0.003648
2018-10-31   -0.002324
2018-11-30    0.002333
2018-12-31   -0.000021
2019-01-31   -0.000109
2019-0